All this is old code from the first summer. See GOES_master

In [22]:
from sunpy.instr.goes import get_goes_event_list
from sunpy.time import parse_time, TimeRange, is_time_in_given_format
t1_s = '2013/05/22 00:00'  
t2_s = '2013/05/23 00:00'

#query the goes event list for all flares before and after the flare peak time of the event you are interested in
g_evt_list = get_goes_event_list(TimeRange(t1_s, t2_s))

In [ ]:
#read 1 minute GOES data

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sunpy.lightcurve import LightCurve
from sunpy.instr.goes import calculate_temperature_em

int_xs = [] #short xray
int_xl = [] #long xray
intflux_s = []
intflux_l = []
t_chianti = []
em_chianti = []
pkflux = []

#for j in range(len(sep_new['Flare Max.'])):
    #if isinstance(sep_new['Flare Max.'][j],str) and isinstance(start_time[j],datetime.date): #check for nan and correct dates
        #flare_dat = sep_new['Flare Max.'][j]
        year = '2015'
        month = '10'

        monthstart = '01'
        leap_years = np.arange(1984,2020,4)
        
        #date for end of the month
        days_31 = ['01','03','05','07','08','10','12']
        days_30 = ['04','06','09','11']

        if (month in days_31) == True:
            monthend = '31'

        elif (month in days_30) == True:
            monthend = '30'

        elif month == '02':
            if (month in leap_years) == True:
                monthend = '29'
            else:
                monthend = '28'
        
        #access url where goes data is kept
        temp_url = "https://satdat.ngdc.noaa.gov/sem/goes/data/avg/"+year+"/"+month+'/'
        res = requests.get(temp_url)
        soup = BeautifulSoup(res.content,'lxml')
        table = soup.find_all('table')
        df = pd.read_html(str(table))[1]
        goes_s =  list(df[0])[-1]
        goes_num = goes_s[4:6]
        path_to_csv = "https://satdat.ngdc.noaa.gov/sem/goes/data/avg/"+year+"/"+month+'/'+goes_s+"csv/g"+goes_num+"_xrs_1m_"+year+month+monthstart+"_"+year+month+monthend+".csv"

        url = path_to_csv

        import urllib
        try: response = urllib.request.urlopen(url)
        except: url = "https://satdat.ngdc.noaa.gov/sem/goes/data/avg/"+year+"/"+month+'/'+list(df[0])[-2]+"csv/g"+list(df[0])[-2][4:6]+"_xrs_1m_"+year+month+monthstart+"_"+year+month+monthend+".csv"
        response = urllib.request.urlopen(url)
            
        data = response.read().decode('utf-8')

        datasplit = data.split('\n')
        
        #put data in readable format and looks for line where xray data begins
        for rownum, row in enumerate(datasplit):
            if row[0:8] == "time_tag":
                nskiprows = rownum + 1
        
        #parsing function
        parser = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")          

        #read in cvs file from url and parse column time_tag with the function parse
        if url[59:61] == '12':
            c = pd.read_csv(url, skiprows=nskiprows, parse_dates=['time_tag'] ,date_parser = parser,names = ['time_tag','xs','xl'])
        else:
            c = pd.read_csv(url, skiprows=nskiprows, parse_dates=['time_tag'] ,date_parser = parser,names = ['time_tag','xs','xl'],usecols = [0,3,6])
    
        #set time_tag to the index
        c.set_index('time_tag',inplace=True)  

        #find just the subset of dates between the flare start time and end time - has to be a date time format
        t1_s = datetime.datetime.strftime(start_time[j], '%Y/%m/%d %H:%M')  
        t2_s = datetime.datetime.strftime(end_time[j], '%Y/%m/%d %H:%M')
        data_subset = c[t1_s:t2_s]  

        #sum over the time - 60 second time intervals
        sum_xrsa = (60 * data_subset['xs']).sum()
        sum_xrsb = (60 * data_subset['xl']).sum()
        
        int_xs.append(sum_xrsa)
        int_xl.append(sum_xrsb)
        
        #integrated flux with background radiation subtracted
        flx_s = (60 * (data_subset['xs'] - data_subset['xs'][0])).sum()
        flx_l = (60 * (data_subset['xl'] - data_subset['xl'][0])).sum()
        
        intflux_s.append(flx_s)
        intflux_l.append(flx_l)
        
        ###temperature and emission measures
        #convert the 1 min GOES data into a GOES lightcurve object - required for temp and em calculation
        lc = LightCurve.create({"xrsa": data_subset['xs'], "xrsb":data_subset['xl']}, index = data_subset.index)

        #add the satellite id to the meta data of the lightcurve object i.e. GOES 10, GOES 11, GOES 12
        name1 = goes_s
        name2 = name1.upper().split('S')
        name3 = name2[0] + 'S ' + name2[1][0:2]
        lc.meta["TELESCOP"] = name3

        #if an error arises and the temp is not in the accpetable range check to see if there are any 
        #negative values equal to -9999.0 in the array. keep only the rows that don't have an exray 
        #flux of -99999 (or whatever value they use that marks a bad value)
        lc.data.xrsa = lc.data.xrsa[lc.data.xrsa != -99999.0]
        lc.data.xrsb = lc.data.xrsb[lc.data.xrsb != -99999.0]

        #calculate the temperature and emission measure
        lc_new = calculate_temperature_em(lc) 

        #flare max temp and em
        temp = lc_new.data.temperature.max()
        em = lc_new.data.em.max()
        
        t_chianti.append(temp)
        em_chianti.append(em)
        
        #flare peak flux 
        flrpk = data_subset['xl'].max()
        pkflux.append(flrpk)
        
    else:
        int_xs.append(nan)
        int_xl.append(nan)
        intflux_s.append(nan)
        intflux_l.append(nan)
        t_chianti.append(nan)
        em_chianti.append(nan)
        pkflux.append(nan)
        
  


